In [1]:
import sys
sys.path.append("/sps/lsst/users/ebarroso/crow")
from crow.cluster_modules.shear_profile import *

In [2]:
import pyccl as ccl
hmf = ccl.halos.MassFuncTinker08(mass_def="200c")
cosmo = ccl.Cosmology(
    Omega_c=0.2607,      # Cold dark matter density
    Omega_b=0.04897,     # Baryon density
    h=0.6766,            # Hubble parameter
    sigma8=0.8102,       # Matter fluctuation amplitude
    n_s=0.9665,          # Spectral index
)
cl_delta_sigma = ClusterShearProfile(cosmo, hmf, 4.0, True)

cl_reduced_shear = ClusterShearProfile(cosmo, hmf, 4.0, False)

cl_reduced_shear_interp = ClusterShearProfile(cosmo, hmf, 4.0, False, True)

cl_reduced_shear_interp_mis = ClusterShearProfile(cosmo, hmf, 4.0, False, False)


### Test values of shear profile

In [3]:
import time

start = time.time()
ds_val = cl_delta_sigma.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Delta sigma:", ds_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear.set_beta_parameters(10)
gt_val = cl_reduced_shear.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear exact:", gt_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear_interp.set_beta_parameters(10)
cl_reduced_shear_interp.set_beta_s_interp(1.1, 1.3)
gt_val = cl_reduced_shear_interp.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear interpolated beta:", gt_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear_interp_mis.set_beta_parameters(10)
cl_reduced_shear_interp_mis.set_beta_s_interp(1.1, 1.3)
cl_reduced_shear_interp_mis.set_miscentering(0.2)
gt_val = cl_reduced_shear_interp_mis.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear interpolated beta with miscentering:", gt_val, " | Time:", end - start, "s")


Delta sigma: [1.13451921e+12]  | Time: 0.0025360584259033203 s
Reduced shear exact: [0.00027987]  | Time: 0.10050415992736816 s
Reduced shear interpolated beta: [0.00027987]  | Time: 0.05008506774902344 s
Reduced shear interpolated beta with miscentering: [0.00022393]  | Time: 0.1774766445159912 s


## Test implementation on the recipe

In [5]:
from crow.recipes.murata_binned_spec_z import MurataBinnedSpecZRecipe
from crow.cluster_modules.mass_proxy import MurataBinned
from crow.cluster_modules.kernel import SpectroscopicRedshift
#from firecrown.models.cluster import ClusterProperty
from crow.properties import ClusterProperty
import time

pivot_mass, pivot_redshift = 14.625862906, 0.6
mass_distribution = MurataBinned(pivot_mass, pivot_redshift)
redshift_distribution = SpectroscopicRedshift()

def run_recipe(label, cluster_theory, mass_proxy_interval1=(1.1, 1.2), mass_proxy_interval2=(0.2, 0.31), redshiftinterval1=(1, 2), redshiftinterval2=(0.86, 1.0)):
    print(f"\n{label} Starts here\n{'-'*50}")
    average_on = ClusterProperty.NONE
    average_on |= ClusterProperty.DELTASIGMA
    recipe = MurataBinnedSpecZRecipe(
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
    )
    t0 = time.time()
    val1 = recipe.evaluate_theory_prediction_shear_profile(mass_proxy_interval1, redshiftinterval1, 1.0, 440, average_on)
    t1 = time.time()
    print(f"Eval 1: {val1:.3e} | Time: {t1-t0:.4f} s")

    t2 = time.time()
    val2 = recipe.evaluate_theory_prediction_shear_profile(mass_proxy_interval2, redshiftinterval2, 4.0, 440, average_on)
    t3 = time.time()
    print(f"Eval 2: {val2:.3e} | Time: {t3-t2:.4f} s")
    t22 = time.time()
    val2 = recipe.evaluate_theory_prediction_shear_profile_grid(mass_proxy_interval2, redshiftinterval2, 4.0, 440, average_on, mass_p=30, z_p=30)
    t33 = time.time()
    print(f"Eval 3 with grid: {val2:.3e} | Time: {t33-t22:.4f} s")
    t4 = time.time()
    counts = recipe.evaluate_theory_prediction_counts((0.2, 0.31), (0.86, 1.0), 440)
    t5 = time.time() 
    print(f"Counts: {counts:.3e} Time: {t5-t4:.4f} s")
    t6 = time.time()   
    counts = recipe.evaluate_theory_prediction_counts_grid((0.2, 0.31), (0.86, 1.0), 440, mass_p=20, z_p=20)
    t7 = time.time()
    print(f"Counts grid: {counts:.3e} Time: {t7-t6:.4f} s")


# Run different scenarios
run_recipe("Delta Sigma", cl_delta_sigma)
run_recipe("Tangential Shear", cl_reduced_shear)
run_recipe("Tangential Shear with Interpolation", cl_reduced_shear_interp)



Delta Sigma Starts here
--------------------------------------------------
Eval 1: 8.249e+15 | Time: 0.7162 s
Eval 2: 1.309e+15 | Time: 0.3789 s
Eval 3: 1.343e+15 | Time: 2.1705 s
Counts: 2.658e+02 Time: 0.0030 s
Counts grid: 2.524e+02 Time: 0.1022 s

Tangential Shear Starts here
--------------------------------------------------
Eval 1: 2.134e+00 | Time: 10.7128 s
Eval 2: 4.438e-01 | Time: 17.3217 s
Eval 3: 4.554e-01 | Time: 25.8178 s
Counts: 2.658e+02 Time: 0.0023 s
Counts grid: 2.524e+02 Time: 0.1004 s

Tangential Shear with Interpolation Starts here
--------------------------------------------------
Eval 1: 2.134e+00 | Time: 0.6044 s
Eval 2: 4.438e-01 | Time: 0.6359 s
Eval 3: 4.554e-01 | Time: 2.3890 s
Counts: 2.658e+02 Time: 0.0021 s
Counts grid: 2.524e+02 Time: 0.1013 s
